## Data Loading

In [7]:
import os
'CV-EmpathNet' in os.listdir('/content/drive/MyDrive')

True

In [8]:
# !rm -rf /content/spect_images
!mkdir spect_images

In [9]:
# !unzip /content/drive/MyDrive/CV-EmpathNet/spectogram_images.zip -d spect_images/
!unzip /content/drive/MyDrive/CV-EmpathNet/spectogram_images_old.zip -d spect_images/

Archive:  /content/drive/MyDrive/CV-EmpathNet/spectogram_images_old.zip
   creating: spect_images/spectogram_images/
  inflating: spect_images/spectogram_images/713_P_spectrogram.png  
  inflating: spect_images/spectogram_images/390_P_spectrogram.png  
  inflating: spect_images/spectogram_images/444_P_spectrogram.png  
  inflating: spect_images/spectogram_images/669_P_spectrogram.png  
  inflating: spect_images/spectogram_images/340_P_spectrogram.png  
  inflating: spect_images/spectogram_images/627_P_spectrogram.png  
  inflating: spect_images/spectogram_images/368_P_spectrogram.png  
  inflating: spect_images/spectogram_images/641_P_spectrogram.png  
  inflating: spect_images/spectogram_images/326_P_spectrogram.png  
  inflating: spect_images/spectogram_images/691_P_spectrogram.png  
  inflating: spect_images/spectogram_images/422_P_spectrogram.png  
  inflating: spect_images/spectogram_images/470_P_spectrogram.png  
  inflating: spect_images/spectogram_images/374_P_spectrogram.png  

In [10]:
#Read The Train and Test csvs
import pandas as pd
trdf = pd.read_csv('/content/drive/MyDrive/CV-EmpathNet/train_text_split.csv')
trdf

,Participant_ID,Gender,PHQ_Binary,PHQ_Score,PCL-C (PTSD),PTSD Severity,Sample,Transcript
0,302,male,0,4,0,28,302,just move around a little bit when you're fin...
1,303,female,0,0,0,17,303,wow okay when you're finished when she's done...
2,304,female,0,6,0,20,304,so we'll just move around a little bit tonight...
3,305,male,0,7,0,28,305,okay looks good so we can just move around a l...
4,307,female,0,4,0,23,307,looking at you all right okay so now let's mak...
...,...,...,...,...,...,...,...,...
158,695,male,0,7,1,62,695,okay I will press the button just push the ...
159,697,male,0,5,0,24,697,okie dokie yeah okay here we go I'm not a t...
160,702,male,0,0,0,19,702,hi I'm not a therapist are you okay yes oka...
161,703,male,0,8,0,28,703,and please are you okay with this yes doing ...


In [11]:
tedf = pd.read_csv('/content/drive/MyDrive/CV-EmpathNet/test_text_split.csv')
tedf

,Participant_ID,Gender,PHQ_Binary,PHQ_Score,PCL-C (PTSD),PTSD Severity,Sample,Transcript
0,600,female,0,5,0,23.0,600,okay there she is coming to go ahead and Shrin...
1,602,female,1,13,1,67.0,602,this is super need I like this me either at a...
2,604,male,1,12,0,30.0,604,I passed out to make sure that our audio recog...
3,605,male,0,2,0,23.0,605,remember to bring up a virtual human for a sec...
4,606,female,0,5,0,46.0,606,hey I just got it I just got a new so okay so ...
5,607,female,0,7,0,29.0,607,bring her up for a quick second here bring he...
6,609,male,0,0,0,19.0,609,research experience actually going to see a v...
7,615,male,0,3,0,22.0,615,I'm going to have you alert me to look up sur...
8,618,male,0,4,0,23.0,618,you live the are you what are you going to g...
9,619,female,0,6,0,37.0,619,that should be fine in the matching to put thi...


In [12]:
tp = trdf.shape[0]
ts = tedf.shape[0]
print('Train Participants:', tp)
print('Test Participants:', ts)
print('Total Participants:', ts+tp)

Train Participants: 163
Test Participants: 56
Total Participants: 219


## Data Preprocessing

In [13]:
# !rm -rf train
# !rm -rf test

In [14]:
#Seperate Images Into Train and Test Folders and 0 and 1 class.
import os

os.makedirs('/content/train/1', exist_ok=True)
os.makedirs('/content/train/0', exist_ok=True)

os.makedirs('/content/test/1', exist_ok=True)
os.makedirs('/content/test/0', exist_ok=True)

In [15]:
from shutil import copy2
src = '/content/spect_images/spectogram_images'
dst = '/content/train/0'
for id in trdf[trdf['PHQ_Binary']==0]['Participant_ID']:
  copy2(os.path.join(src, str(id) +'_P_spectrogram.png'), dst)

dst = '/content/train/1'
for id in trdf[trdf['PHQ_Binary']==1]['Participant_ID']:
  copy2(os.path.join(src, str(id) +'_P_spectrogram.png'), dst)

dst = '/content/test/1'
for id in tedf[tedf['PHQ_Binary']==1]['Participant_ID']:
  copy2(os.path.join(src, str(id) +'_P_spectrogram.png'), dst)

dst = '/content/test/0'
for id in tedf[tedf['PHQ_Binary']==0]['Participant_ID']:
  copy2(os.path.join(src, str(id) +'_P_spectrogram.png'), dst)


In [16]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
#Preliminary Train and Test ImageDataGenerators
preproc_func = tf.keras.applications.densenet.preprocess_input
# preproc_func = tf.keras.applications.xception.preprocess_input

In [19]:
#batch size
bs = 4
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preproc_func)
train_gen = train_datagen.flow_from_directory('/content/train', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)
# train_gen = train_datagen.flow_from_directory('/content/train', (224,224), class_mode = 'categorical', batch_size = bs, shuffle=True, seed=42)

print('Class Indices')
print(train_gen.class_indices)

Found 163 images belonging to 2 classes.
Class Indices
{'0': 0, '1': 1}


In [20]:
#batch size
bs = 4
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preproc_func)
# test_gen = train_datagen.flow_from_directory('/content/test', (224,224), class_mode = 'categorical', batch_size = bs, shuffle=True, seed=42)
test_gen = train_datagen.flow_from_directory('/content/test', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)

print('Class Indices')
print(test_gen.class_indices)

Found 56 images belonging to 2 classes.
Class Indices
{'0': 0, '1': 1}


## Model Selection

In [ ]:
##################################################################################################################
# The following article was referred to for writing the below lines of code
# https://towardsdatascience.com/how-to-choose-the-best-keras-pre-trained-model-for-image-classification-b850ca4428d4


In [ ]:
#Create a model dictionary
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}

In [ ]:
model_dictionary

{'DenseNet121': <function keras.applications.densenet.DenseNet121>,
 'DenseNet169': <function keras.applications.densenet.DenseNet169>,
 'DenseNet201': <function keras.applications.densenet.DenseNet201>,
 'EfficientNetB0': <function keras.applications.efficientnet.EfficientNetB0>,
 'EfficientNetB1': <function keras.applications.efficientnet.EfficientNetB1>,
 'EfficientNetB2': <function keras.applications.efficientnet.EfficientNetB2>,
 'EfficientNetB3': <function keras.applications.efficientnet.EfficientNetB3>,
 'EfficientNetB4': <function keras.applications.efficientnet.EfficientNetB4>,
 'EfficientNetB5': <function keras.applications.efficientnet.EfficientNetB5>,
 'EfficientNetB6': <function keras.applications.efficientnet.EfficientNetB6>,
 'EfficientNetB7': <function keras.applications.efficientnet.EfficientNetB7>,
 'EfficientNetV2B0': <function keras.applications.efficientnet_v2.EfficientNetV2B0>,
 'EfficientNetV2B1': <function keras.applications.efficientnet_v2.EfficientNetV2B1>,
 '

In [ ]:
print('No. of models considered = ',len(list(model_dictionary.keys())))

No. of models considered =  35


In [ ]:
# Loop over each model available in Keras
from tensorflow.python.keras.layers import Dense, Flatten

model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': [], 'train_accuracy': [], 'train_loss':[], 'val_loss':[]}

n_epochs = 10

num_train = trdf.shape[0]
num_iterations = int(num_train/bs)

error_models = []
count = 1
for model_name, model in tqdm(model_dictionary.items()):
    print('-'*50)
    print(count,'.',model_name)
        
    # load the pre-trained model with global average pooling as the last layer and freeze the model weights
    
    input_shape = (224,224,3)

    try:
      pre_trained_model = model(include_top=False, input_shape=input_shape)
      pre_trained_model.trainable = False
      
      clf_model = tf.keras.models.Sequential()
      clf_model.add(pre_trained_model)

      clf_model.add(Flatten())

      clf_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
      clf_model.compile(loss='binary_crossentropy', metrics=['accuracy'])

      history = clf_model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations)
      
      # Calculate all relevant metrics
      model_benchmarks['model_name'].append(model_name)
      model_benchmarks['num_model_params'].append(pre_trained_model.count_params())
      model_benchmarks['validation_accuracy'].append(history.history['val_accuracy'][-1])
      model_benchmarks['train_accuracy'].append(history.history['accuracy'][-1])
      model_benchmarks['train_loss'].append(history.history['loss'][-1])
      model_benchmarks['val_loss'].append(history.history['val_loss'][-1])
      
      print('Model Trained', '-'*30)

    except Exception as e:
      print('!'*50)
      print('Error',e)
      error_models.append(model_name)
      clf_model.summary()
      print('!'*50)
      # break

    count += 1


  0%|          | 0/35 [00:00<?, ?it/s]

--------------------------------------------------
1 . DenseNet121
Epoch 1/10
40/40 [==============================] - 8s 65ms/step - loss: 1.3915 - accuracy: 0.6855 - val_loss: 0.6231 - val_accuracy: 0.6964
Epoch 2/10
40/40 [==============================] - 1s 24ms/step - loss: 1.0867 - accuracy: 0.6604 - val_loss: 0.6145 - val_accuracy: 0.6964
Epoch 3/10
40/40 [==============================] - 1s 24ms/step - loss: 1.0705 - accuracy: 0.6855 - val_loss: 0.7287 - val_accuracy: 0.6964
Epoch 4/10
40/40 [==============================] - 1s 24ms/step - loss: 1.0633 - accuracy: 0.6730 - val_loss: 0.6572 - val_accuracy: 0.6964
Epoch 5/10
40/40 [==============================] - 1s 25ms/step - loss: 1.1092 - accuracy: 0.6730 - val_loss: 1.2489 - val_accuracy: 0.6964
Epoch 6/10
40/40 [==============================] - 1s 24ms/step - loss: 0.9954 - accuracy: 0.6918 - val_loss: 0.9399 - val_accuracy: 0.3036
Epoch 7/10
40/40 [==============================] - 1s 24ms/step - loss: 1.0855 - accur

  3%|▎         | 1/35 [00:19<11:15, 19.88s/it]

Model Added **************************************************
--------------------------------------------------
2 . DenseNet169
Epoch 1/10
40/40 [==============================] - 11s 89ms/step - loss: 1.6720 - accuracy: 0.6855 - val_loss: 1.8633 - val_accuracy: 0.6964
Epoch 2/10
40/40 [==============================] - 1s 33ms/step - loss: 1.3582 - accuracy: 0.6289 - val_loss: 2.8335 - val_accuracy: 0.6964
Epoch 3/10
40/40 [==============================] - 1s 33ms/step - loss: 1.3364 - accuracy: 0.7170 - val_loss: 2.1316 - val_accuracy: 0.6964
Epoch 4/10
40/40 [==============================] - 1s 32ms/step - loss: 1.5456 - accuracy: 0.6478 - val_loss: 0.6888 - val_accuracy: 0.5179
Epoch 5/10
40/40 [==============================] - 1s 32ms/step - loss: 1.3975 - accuracy: 0.6604 - val_loss: 2.7666 - val_accuracy: 0.6964
Epoch 6/10
40/40 [==============================] - 1s 33ms/step - loss: 1.5117 - accuracy: 0.6289 - val_loss: 0.7084 - val_accuracy: 0.6964
Epoch 7/10
40/40 [=====

  6%|▌         | 2/35 [00:46<13:12, 24.01s/it]

...

Model Added **************************************************
--------------------------------------------------
7 . EfficientNetB3
Epoch 1/10
40/40 [==============================] - 13s 78ms/step - loss: 5.1349 - accuracy: 0.6478 - val_loss: 6.6876 - val_accuracy: 0.6964
Epoch 2/10
40/40 [==============================] - 1s 26ms/step - loss: 3.9395 - accuracy: 0.6667 - val_loss: 3.7003 - val_accuracy: 0.6964
Epoch 3/10
40/40 [==============================] - 1s 27ms/step - loss: 4.1553 - accuracy: 0.6164 - val_loss: 11.1215 - val_accuracy: 0.6964
Epoch 4/10
40/40 [==============================] - 1s 27ms/step - loss: 4.3245 - accuracy: 0.6918 - val_loss: 0.6278 - val_accuracy: 0.6964
Epoch 5/10
40/40 [==============================] - 1s 27ms/step - loss: 4.7260 - accuracy: 0.6164 - val_loss: 0.6394 - val_accuracy: 0.6964
Epoch 6/10
40/40 [==============================] - 1s 27ms/step - loss: 3.9926 - accuracy: 0.6918 - val_loss: 1.3658 - val_accuracy: 0.6964
Epoch 7/10
40/40 [=

 20%|██        | 7/35 [02:46<11:04, 23.74s/it]

Model Added **************************************************
--------------------------------------------------
8 . EfficientNetB4


 20%|██        | 7/35 [02:49<11:19, 24.25s/it]


KeyboardInterrupt: ignored

In [ ]:
# Convert Results to DataFrame for easy viewing
benchmark_df = pd.DataFrame(model_benchmarks)

# sort in ascending order of num_model_params column
benchmark_df.sort_values('num_model_params', inplace=True)

# write results to csv file
# benchmark_df.to_csv('/content/drive/MyDrive/CV-EmpathNet/benchmark_df.csv', index=False)
benchmark_df.to_csv('/content/drive/MyDrive/CV-EmpathNet/benchmark_new_df.csv', index=False)
benchmark_df

,model_name,num_model_params,validation_accuracy,train_accuracy,train_loss,val_loss
23,MobileNetV3Small,939120,0.696429,0.610063,1.261865,0.641164
21,MobileNetV2,2257984,0.696429,0.666667,3.931767,4.132025
22,MobileNetV3Large,2996352,0.696429,0.641509,1.020504,1.215575
20,MobileNet,3228864,0.696429,0.735849,0.772681,0.838050
3,EfficientNetB0,4049571,0.696429,0.654088,2.758051,4.060661
24,NASNetMobile,4269716,0.696429,0.660377,1.742669,2.048274
11,EfficientNetV2B0,5919312,0.696429,0.616352,3.054909,1.487831
4,EfficientNetB1,6575239,0.696429,0.628931,3.755259,5.526180
12,EfficientNetV2B1,6931124,0.696429,0.679245,2.554912,1.696293
0,DenseNet121,7037504,0.642857,0.679245,0.963008,0.653126


In [ ]:
benchmark_df.sort_values('validation_accuracy')

,model_name,num_model_params,validation_accuracy,train_accuracy,train_loss,val_loss
27,ResNet152,58370944,0.303571,0.610063,2.159722,0.907543
16,EfficientNetV2M,53150388,0.303571,0.679245,5.956095,8.798914
2,DenseNet201,18321984,0.446429,0.666667,1.063560,0.807484
0,DenseNet121,7037504,0.642857,0.679245,0.963008,0.653126
17,EfficientNetV2S,20331360,0.696429,0.622642,7.168417,9.962553
33,Xception,20861480,0.696429,0.767296,0.545874,0.877682
19,InceptionV3,21802784,0.696429,0.691824,0.878346,1.367819
30,ResNet50V2,23564800,0.696429,0.710692,0.727831,1.128280
29,ResNet50,23587712,0.696429,0.641509,2.267992,2.204662
23,MobileNetV3Small,939120,0.696429,0.610063,1.261865,0.641164


In [ ]:
benchmark_df.sort_values('train_accuracy')

,model_name,num_model_params,validation_accuracy,train_accuracy,train_loss,val_loss
25,ResNet101,42658176,0.696429,0.559748,2.067032,3.175153
14,EfficientNetV2B3,12930622,0.696429,0.603774,5.956751,5.856483
23,MobileNetV3Small,939120,0.696429,0.610063,1.261865,0.641164
27,ResNet152,58370944,0.303571,0.610063,2.159722,0.907543
8,EfficientNetB5,28513527,0.696429,0.610063,10.997963,13.233499
7,EfficientNetB4,17673823,0.696429,0.610063,6.457074,12.464493
11,EfficientNetV2B0,5919312,0.696429,0.616352,3.054909,1.487831
17,EfficientNetV2S,20331360,0.696429,0.622642,7.168417,9.962553
9,EfficientNetB6,40960143,0.696429,0.628931,15.089182,0.647317
4,EfficientNetB1,6575239,0.696429,0.628931,3.755259,5.526180


In [ ]:
benchmark_df.sort_values('val_loss')

,model_name,num_model_params,validation_accuracy,train_accuracy,train_loss,val_loss
23,MobileNetV3Small,939120,0.696429,0.610063,1.261865,0.641164
9,EfficientNetB6,40960143,0.696429,0.628931,15.089182,0.647317
0,DenseNet121,7037504,0.642857,0.679245,0.963008,0.653126
2,DenseNet201,18321984,0.446429,0.666667,1.063560,0.807484
20,MobileNet,3228864,0.696429,0.735849,0.772681,0.838050
33,Xception,20861480,0.696429,0.767296,0.545874,0.877682
27,ResNet152,58370944,0.303571,0.610063,2.159722,0.907543
31,VGG16,14714688,0.696429,0.660377,0.753402,0.936005
18,InceptionResNetV2,54336736,0.696429,0.723270,0.674017,0.963250
28,ResNet152V2,58331648,0.696429,0.716981,0.693627,1.022689


In [ ]:
benchmark_df.sort_values('train_loss')

,model_name,num_model_params,validation_accuracy,train_accuracy,train_loss,val_loss
33,Xception,20861480,0.696429,0.767296,0.545874,0.877682
26,ResNet101V2,42626560,0.696429,0.754717,0.666170,1.286456
18,InceptionResNetV2,54336736,0.696429,0.723270,0.674017,0.963250
28,ResNet152V2,58331648,0.696429,0.716981,0.693627,1.022689
30,ResNet50V2,23564800,0.696429,0.710692,0.727831,1.128280
31,VGG16,14714688,0.696429,0.660377,0.753402,0.936005
20,MobileNet,3228864,0.696429,0.735849,0.772681,0.838050
32,VGG19,20024384,0.696429,0.716981,0.851402,1.660126
19,InceptionV3,21802784,0.696429,0.691824,0.878346,1.367819
0,DenseNet121,7037504,0.642857,0.679245,0.963008,0.653126


In [ ]:
# The following article was referred to for writing the above lines of code
# https://towardsdatascience.com/how-to-choose-the-best-keras-pre-trained-model-for-image-classification-b850ca4428d4
##################################################################################################################


## MobileNet V2

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
preproc_func = tensorflow.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
#batch size
bs = 32
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preproc_func)
train_gen = train_datagen.flow_from_directory('/content/train', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)

print('Class Indices')
print(train_gen.class_indices)

Found 163 images belonging to 2 classes.
Class Indices
{'0': 0, '1': 1}


In [ ]:
#batch size
bs = 32
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preproc_func)
test_gen = train_datagen.flow_from_directory('/content/test', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)

print('Class Indices')
print(test_gen.class_indices)

Found 56 images belonging to 2 classes.
Class Indices
{'0': 0, '1': 1}


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# num_classes = 2

pretrained_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3))
pretrained_model.trainable = False

model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
optim = tf.keras.optimizers.Adam(
    # learning_rate=0.001,
    # learning_rate=0.00001,
    learning_rate=0.000001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"
)

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optim)

In [ ]:
#Whole Model Trainable
for layer in model.layers:
  layer.trainable = True

#Whole Model Trainable
for layer in model.layers:
  print(layer.trainable)

True
True
True


In [ ]:
#With 0.0001 lr.
n_epochs = 1000
num_train = trdf.shape[0]
num_iterations = int(num_train/bs)

history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations)

Epoch 1/1000
5/5 [==============================] - 11s 293ms/step - loss: 0.9759 - accuracy: 0.2313 - val_loss: 0.8950 - val_accuracy: 0.3036
Epoch 2/1000
5/5 [==============================] - 0s 96ms/step - loss: 0.9352 - accuracy: 0.2250 - val_loss: 0.8590 - val_accuracy: 0.3036
Epoch 3/1000
5/5 [==============================] - 0s 97ms/step - loss: 0.8876 - accuracy: 0.2366 - val_loss: 0.8254 - val_accuracy: 0.3036
Epoch 4/1000
5/5 [==============================] - 0s 88ms/step - loss: 0.8554 - accuracy: 0.2214 - val_loss: 0.7947 - val_accuracy: 0.3036
Epoch 5/1000
5/5 [==============================] - 0s 90ms/step - loss: 0.8056 - accuracy: 0.2519 - val_loss: 0.7677 - val_accuracy: 0.3036
Epoch 6/1000
5/5 [==============================] - 0s 96ms/step - loss: 0.7815 - accuracy: 0.2313 - val_loss: 0.7434 - val_accuracy: 0.3036
Epoch 7/1000
5/5 [==============================] - 0s 85ms/step - loss: 0.7558 - accuracy: 0.2214 - val_loss: 0.7219 - val_accuracy: 0.3036
Epoch 8/100

In [ ]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

#With 0.001 lr.
n_epochs = 100
num_train = trdf.shape[0]
num_iterations = int(num_train/bs)

history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations)

Epoch 1/100
5/5 [==============================] - 12s 298ms/step - loss: 2.5252 - accuracy: 0.2137 - val_loss: 1.1788 - val_accuracy: 0.3036
Epoch 2/100
5/5 [==============================] - 0s 87ms/step - loss: 0.8841 - accuracy: 0.2137 - val_loss: 0.8746 - val_accuracy: 0.3036
Epoch 3/100
5/5 [==============================] - 0s 105ms/step - loss: 0.5555 - accuracy: 0.2061 - val_loss: 0.9770 - val_accuracy: 0.3036
Epoch 4/100
5/5 [==============================] - 0s 92ms/step - loss: 0.8629 - accuracy: 0.2366 - val_loss: 1.1501 - val_accuracy: 0.3036
Epoch 5/100
5/5 [==============================] - 0s 89ms/step - loss: 0.9240 - accuracy: 0.2519 - val_loss: 1.1078 - val_accuracy: 0.3036
Epoch 6/100
5/5 [==============================] - 0s 89ms/step - loss: 0.7571 - accuracy: 0.2519 - val_loss: 0.6849 - val_accuracy: 0.3036
Epoch 7/100
5/5 [==============================] - 0s 87ms/step - loss: 0.5780 - accuracy: 0.2137 - val_loss: 0.6061 - val_accuracy: 0.3036
Epoch 8/100
5/5 [

## Xception

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
preproc_func = tensorflow.keras.applications.xception.preprocess_input

In [ ]:
#batch size
bs = 32
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preproc_func)
train_gen = train_datagen.flow_from_directory('/content/train', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)

print('Class Indices')
print(train_gen.class_indices)

Found 163 images belonging to 2 classes.
Class Indices
{'0': 0, '1': 1}


In [ ]:
#batch size
bs = 32
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preproc_func)
test_gen = train_datagen.flow_from_directory('/content/test', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)

print('Class Indices')
print(test_gen.class_indices)

Found 56 images belonging to 2 classes.
Class Indices
{'0': 0, '1': 1}


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.xception import Xception


pretrained_model = Xception(include_top=False, weights='imagenet', input_shape=(224,224,3))
pretrained_model.trainable = False
model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
optim = tf.keras.optimizers.Adam(
    # learning_rate=0.001,
    # learning_rate=0.00001,
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"
)

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optim)

In [ ]:
#Whole Model Trainable
for layer in model.layers:
  layer.trainable = True

#Whole Model Trainable
for layer in model.layers:
  print(layer.trainable)

True
True
True


In [ ]:
#With 0.0001 lr.
n_epochs = 1000
num_train = trdf.shape[0]
num_iterations = int(num_train/bs)

history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations)

Epoch 1/1000
5/5 [==============================] - 4s 351ms/step - loss: 0.6495 - accuracy: 0.6336 - val_loss: 0.6188 - val_accuracy: 0.6964
Epoch 2/1000
5/5 [==============================] - 1s 127ms/step - loss: 0.5437 - accuracy: 0.7863 - val_loss: 0.6212 - val_accuracy: 0.6964
Epoch 3/1000
5/5 [==============================] - 1s 129ms/step - loss: 0.5089 - accuracy: 0.7939 - val_loss: 0.6554 - val_accuracy: 0.6964
Epoch 4/1000
5/5 [==============================] - 1s 116ms/step - loss: 0.5631 - accuracy: 0.7634 - val_loss: 0.6747 - val_accuracy: 0.6964
Epoch 5/1000
5/5 [==============================] - 1s 141ms/step - loss: 0.5898 - accuracy: 0.7481 - val_loss: 0.6548 - val_accuracy: 0.6964
Epoch 6/1000
5/5 [==============================] - 1s 144ms/step - loss: 0.4974 - accuracy: 0.8015 - val_loss: 0.6468 - val_accuracy: 0.6964
Epoch 7/1000
5/5 [==============================] - 1s 132ms/step - loss: 0.5353 - accuracy: 0.7750 - val_loss: 0.6351 - val_accuracy: 0.6964
Epoch 

KeyboardInterrupt: ignored

In [ ]:
optim = tf.keras.optimizers.Adam(
    # learning_rate=0.001,
    # learning_rate=0.00001,
    learning_rate=0.00001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"
)

# model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optim)

In [ ]:
#With 0.00001 lr.
n_epochs = 1000
num_train = trdf.shape[0]
num_iterations = int(num_train/bs)

history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations)

Epoch 1/1000
5/5 [==============================] - 8s 588ms/step - loss: 0.8132 - accuracy: 0.4962 - val_loss: 0.6125 - val_accuracy: 0.6964
Epoch 2/1000
5/5 [==============================] - 2s 353ms/step - loss: 0.6320 - accuracy: 0.6718 - val_loss: 0.6344 - val_accuracy: 0.6964
Epoch 3/1000
5/5 [==============================] - 2s 353ms/step - loss: 0.4760 - accuracy: 0.7786 - val_loss: 0.6474 - val_accuracy: 0.6964
Epoch 4/1000
5/5 [==============================] - 2s 352ms/step - loss: 0.4406 - accuracy: 0.7863 - val_loss: 0.6495 - val_accuracy: 0.6964
Epoch 5/1000
5/5 [==============================] - 2s 355ms/step - loss: 0.3870 - accuracy: 0.8244 - val_loss: 0.6401 - val_accuracy: 0.6964
Epoch 6/1000
5/5 [==============================] - 2s 352ms/step - loss: 0.3392 - accuracy: 0.8244 - val_loss: 0.6363 - val_accuracy: 0.6964
Epoch 7/1000
5/5 [==============================] - 2s 432ms/step - loss: 0.3277 - accuracy: 0.8625 - val_loss: 0.6260 - val_accuracy: 0.6964
Epoch 

KeyboardInterrupt: ignored

In [ ]:
# model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

# #With 0.001 lr.
# n_epochs = 100
# num_train = trdf.shape[0]
# num_iterations = int(num_train/bs)

# history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations)

Epoch 1/100
5/5 [==============================] - 12s 298ms/step - loss: 2.5252 - accuracy: 0.2137 - val_loss: 1.1788 - val_accuracy: 0.3036
Epoch 2/100
5/5 [==============================] - 0s 87ms/step - loss: 0.8841 - accuracy: 0.2137 - val_loss: 0.8746 - val_accuracy: 0.3036
Epoch 3/100
5/5 [==============================] - 0s 105ms/step - loss: 0.5555 - accuracy: 0.2061 - val_loss: 0.9770 - val_accuracy: 0.3036
Epoch 4/100
5/5 [==============================] - 0s 92ms/step - loss: 0.8629 - accuracy: 0.2366 - val_loss: 1.1501 - val_accuracy: 0.3036
Epoch 5/100
5/5 [==============================] - 0s 89ms/step - loss: 0.9240 - accuracy: 0.2519 - val_loss: 1.1078 - val_accuracy: 0.3036
Epoch 6/100
5/5 [==============================] - 0s 89ms/step - loss: 0.7571 - accuracy: 0.2519 - val_loss: 0.6849 - val_accuracy: 0.3036
Epoch 7/100
5/5 [==============================] - 0s 87ms/step - loss: 0.5780 - accuracy: 0.2137 - val_loss: 0.6061 - val_accuracy: 0.3036
Epoch 8/100
5/5 [

## Data Augmentation

In [15]:
#Load Numpy Array Of Images.
import numpy as np

train_array = np.load(r'/content/drive/MyDrive/CV-EmpathNet/train_text.npy')
train_array.shape

(163, 768)

In [16]:
#Load Numpy Array Of Images.
import numpy as np

train_array = np.load(r'/content/drive/MyDrive/CV-EmpathNet/train_image.npy')
train_array.shape

(163, 224, 224)

In [17]:
#Load Numpy Array Of Images.
import numpy as np

train_array = np.load(r'/content/drive/MyDrive/CV-EmpathNet/train_image_3_channels.npy')
train_array.shape

(163, 288, 432, 3)

In [18]:
!pip install wand
!apt-get install libmagickwand-dev


     |████████████████████████████████| 139 kB 5.0 MB/s 


In [20]:
# !rm -rf spect_aug_noise

In [21]:
# !rm -rf train_aug_noise
# !rm -rf test_aug_noise

In [22]:
#Create New Augmented Dataset Folders
!cp -r train train_aug_noise
!cp -r test test_aug_noise

In [29]:
#Increase the number of positive (depressed) class samples using gaussian noise injection in the dataset.

from shutil import copy2
from wand.image import Image

# src = '/content/train/0'
# dst = '/content/train_aug_noise/0'
# for id in trdf[trdf['PHQ_Binary']==0]['Participant_ID']:
#   with Image(filename = os.path.join(src, str(id) +'_P_spectrogram.png')) as img:
#     img.noise("gaussian")
#     img.save(filename = os.path.join(dst, str(id) +'aug_P_spectrogram.png'))

src = '/content/train/1'
dst = '/content/train_aug_noise/1'
for id in trdf[trdf['PHQ_Binary']==1]['Participant_ID']:
  with Image(filename = os.path.join(src, str(id) +'_P_spectrogram.png')) as img:
    img.noise("gaussian")
    img.save(filename = os.path.join(dst, str(id) +'aug_P_spectrogram.png'))

src = '/content/test/1'
dst = '/content/test_aug_noise/1'
for id in tedf[tedf['PHQ_Binary']==1]['Participant_ID']:
  with Image(filename = os.path.join(src, str(id) +'_P_spectrogram.png')) as img:
    img.noise("gaussian")
    img.save(filename = os.path.join(dst, str(id) +'aug_P_spectrogram.png'))

# src = '/content/test/0'
# dst = '/content/test_aug_noise/0'
# for id in tedf[tedf['PHQ_Binary']==0]['Participant_ID']:
#   with Image(filename = os.path.join(src, str(id) +'_P_spectrogram.png')) as img:
#     img.noise("gaussian")
#     img.save(filename = os.path.join(dst, str(id) +'aug_P_spectrogram.png'))


In [31]:
# https://medium.com/@makcedward/data-augmentation-for-audio-76912b01fdf6
# https://towardsdatascience.com/audio-deep-learning-made-simple-part-3-data-preparation-and-augmentation-24c6e1f6b52

In [32]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [33]:
preproc_func = tensorflow.keras.applications.xception.preprocess_input

In [36]:
#batch size
bs = 32
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preproc_func)
train_gen = train_datagen.flow_from_directory('/content/train_aug_noise', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)
# train_gen = train_datagen.flow_from_directory('/content/train', (224,224), class_mode = 'categorical', batch_size = bs, shuffle=True, seed=42)

print('Class Indices')
print(train_gen.class_indices)

Found 200 images belonging to 2 classes.
Class Indices
{'0': 0, '1': 1}


In [37]:
#batch size
bs = 32
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preproc_func)
test_gen = test_datagen.flow_from_directory('/content/test', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)

print('Class Indices')
print(test_gen.class_indices)

Found 56 images belonging to 2 classes.
Class Indices
{'0': 0, '1': 1}


# Experimenting with Custom CNNs

In [40]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras import Sequential

#create model

model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(224,224,3)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(10, activation=softmax))
model.add(Dense(1, activation='sigmoid'))

In [42]:
p = len(os.listdir('/content/test/1'))
n = len(os.listdir('/content/test/0'))
print(n/(p+n))

0.6964285714285714


In [43]:
p = len(os.listdir('/content/train/1'))
n = len(os.listdir('/content/train/0'))
print(n/(p+n))

0.7730061349693251


In [44]:
#Whole Model Trainable
for layer in model.layers:
  layer.trainable = True

#Whole Model Trainable
for layer in model.layers:
  print(layer.trainable)

True
True
True
True


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath_r = 'weights_best_rmodel.hdf5'
filepath_a = 'weights_best_amodel.hdf5'
checkpoint1 = ModelCheckpoint(filepath_r,monitor = 'val_recall_15',verbose = 1, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath_a,monitor = 'val_accuracy',verbose = 1, save_best_only=True, mode='max')

checkpoint3 = ModelCheckpoint(filepath_r,monitor = 'accuracy',verbose = 1, save_best_only=True, mode='max')
checkpoint4 = ModelCheckpoint(filepath_a,monitor = 'recall_15',verbose = 1, save_best_only=True, mode='max')

In [ ]:
# With 0.001 lr.

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

n_epochs = 100
num_train = trdf.shape[0]
num_iterations = int(num_train/bs)

history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations)

Epoch 1/100
5/5 [==============================] - 12s 298ms/step - loss: 2.5252 - accuracy: 0.2137 - val_loss: 1.1788 - val_accuracy: 0.3036
Epoch 2/100
5/5 [==============================] - 0s 87ms/step - loss: 0.8841 - accuracy: 0.2137 - val_loss: 0.8746 - val_accuracy: 0.3036
Epoch 3/100
5/5 [==============================] - 0s 105ms/step - loss: 0.5555 - accuracy: 0.2061 - val_loss: 0.9770 - val_accuracy: 0.3036
Epoch 4/100
5/5 [==============================] - 0s 92ms/step - loss: 0.8629 - accuracy: 0.2366 - val_loss: 1.1501 - val_accuracy: 0.3036
Epoch 5/100
5/5 [==============================] - 0s 89ms/step - loss: 0.9240 - accuracy: 0.2519 - val_loss: 1.1078 - val_accuracy: 0.3036
Epoch 6/100
5/5 [==============================] - 0s 89ms/step - loss: 0.7571 - accuracy: 0.2519 - val_loss: 0.6849 - val_accuracy: 0.3036
Epoch 7/100
5/5 [==============================] - 0s 87ms/step - loss: 0.5780 - accuracy: 0.2137 - val_loss: 0.6061 - val_accuracy: 0.3036
Epoch 8/100
5/5 [

In [ ]:
#With 0.0001 lr

optim = tf.keras.optimizers.Adam(
    # learning_rate=0.001,
    # learning_rate=0.00001,
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"
)


model.compile(loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Recall()], optimizer=optim)
n_epochs = 1000
num_train = trdf.shape[0]
num_iterations = int(num_train/bs)

history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations)

Epoch 1/1000
5/5 [==============================] - 2s 193ms/step - loss: 0.6886 - accuracy: 0.6313 - recall_1: 0.2188 - val_loss: 0.6901 - val_accuracy: 0.5342 - val_recall_1: 0.0000e+00
Epoch 2/1000
5/5 [==============================] - 1s 112ms/step - loss: 0.6694 - accuracy: 0.6397 - recall_1: 0.0000e+00 - val_loss: 0.7268 - val_accuracy: 0.5342 - val_recall_1: 0.0000e+00
Epoch 3/1000
5/5 [==============================] - 1s 114ms/step - loss: 0.6742 - accuracy: 0.6187 - recall_1: 0.0000e+00 - val_loss: 0.7304 - val_accuracy: 0.5342 - val_recall_1: 0.0000e+00
Epoch 4/1000
5/5 [==============================] - 1s 108ms/step - loss: 0.6570 - accuracy: 0.6397 - recall_1: 0.0000e+00 - val_loss: 0.7149 - val_accuracy: 0.5342 - val_recall_1: 0.0000e+00
Epoch 5/1000
5/5 [==============================] - 1s 101ms/step - loss: 0.6603 - accuracy: 0.6250 - recall_1: 0.0000e+00 - val_loss: 0.7143 - val_accuracy: 0.5342 - val_recall_1: 0.0000e+00
Epoch 6/1000
5/5 [==========================

KeyboardInterrupt: ignored

# Data Augmentation - SpectAug + Gaussian Noise Injection

In [21]:
#Load Numpy Array Of Images.
import numpy as np

train_array = np.load(r'/content/drive/MyDrive/CV-EmpathNet/train_text.npy')
train_array.shape

(163, 768)

In [22]:
#Load Numpy Array Of Images.
import numpy as np

train_array = np.load(r'/content/drive/MyDrive/CV-EmpathNet/train_image.npy')
train_array.shape

(163, 224, 224)

In [23]:
#Load Numpy Array Of Images.
import numpy as np

train_array = np.load(r'/content/drive/MyDrive/CV-EmpathNet/train_image_3_channels.npy')
train_array.shape

(163, 288, 432, 3)

In [24]:
!pip install wand
!apt-get install libmagickwand-dev


     |████████████████████████████████| 139 kB 14.3 MB/s 


In [26]:
# !rm -rf spect_aug_noise

In [27]:
# !rm -rf train_aug_noise
# !rm -rf test_aug_noise

In [28]:
!cp -r train train_aug_noise
!cp -r test test_aug_noise

In [50]:
#Function Too Perform SpecAugment Strategy, placing random vertical and horizontal masks on the frequency and time domain.

import cv2
import random
from wand.image import Image


src = '/content/train/0/302_P_spectrogram.png'
dst = 'aug_img.jpg'
n_vertical = 1
nv_width = 1
n_hor = 1
nh_width = 1

def aug(src, dst, n_vertical = 1, nv_width = 1, n_hor = 1, nh_width=1):
  img = cv2.imread(src)

  for i in range(n_vertical):
    start_i = random.randint(0,224-nv_width)
    img[:,start_i:start_i+nv_width,:] = 0

  for i in range(n_hor):
    start_i = random.randint(0,224-nh_width)
    img[start_i:start_i+nh_width,:,:] = 0

  cv2.imwrite(dst, img)


aug(src,dst)



In [35]:
#SpecAgument

#Increase the number of positive (depressed) class samples by using gaussian noise injection and SpecAugment like data augmentation techniques on them.
from shutil import copy2


# src = '/content/train/0'
# dst = '/content/train_aug_noise/0'
# for id in trdf[trdf['PHQ_Binary']==0]['Participant_ID']:
#   with Image(filename = os.path.join(src, str(id) +'_P_spectrogram.png')) as img:
#     img.noise("gaussian")
#     img.save(filename = os.path.join(dst, str(id) +'aug_P_spectrogram.png'))

src = '/content/train/1'
dst = '/content/train_aug_noise/1'
for id in trdf[trdf['PHQ_Binary']==1]['Participant_ID']:
  aug(os.path.join(src, str(id) +'_P_spectrogram.png'),os.path.join(dst, str(id) +'saug_P_spectrogram.png'))

src = '/content/test/1'
dst = '/content/test_aug_noise/1'
for id in tedf[tedf['PHQ_Binary']==1]['Participant_ID']:
  aug(os.path.join(src, str(id) +'_P_spectrogram.png'),os.path.join(dst, str(id) +'saug_P_spectrogram.png'))


# src = '/content/test/0'
# dst = '/content/test_aug_noise/0'
# for id in tedf[tedf['PHQ_Binary']==0]['Participant_ID']:
#   with Image(filename = os.path.join(src, str(id) +'_P_spectrogram.png')) as img:
#     img.noise("gaussian")
#     img.save(filename = os.path.join(dst, str(id) +'aug_P_spectrogram.png'))


In [36]:
#Adding Gaussian Noise

from shutil import copy2


# src = '/content/train/0'
# dst = '/content/train_aug_noise/0'
# for id in trdf[trdf['PHQ_Binary']==0]['Participant_ID']:
#   with Image(filename = os.path.join(src, str(id) +'_P_spectrogram.png')) as img:
#     img.noise("gaussian")
#     img.save(filename = os.path.join(dst, str(id) +'aug_P_spectrogram.png'))

src = '/content/train/1'
dst = '/content/train_aug_noise/1'
for id in trdf[trdf['PHQ_Binary']==1]['Participant_ID']:
  with Image(filename = os.path.join(src, str(id) +'_P_spectrogram.png')) as img:
    img.noise("gaussian")
    img.save(filename = os.path.join(dst, str(id) +'aug_P_spectrogram.png'))

src = '/content/test/1'
dst = '/content/test_aug_noise/1'
for id in tedf[tedf['PHQ_Binary']==1]['Participant_ID']:
  with Image(filename = os.path.join(src, str(id) +'_P_spectrogram.png')) as img:
    img.noise("gaussian")
    img.save(filename = os.path.join(dst, str(id) +'aug_P_spectrogram.png'))

# src = '/content/test/0'
# dst = '/content/test_aug_noise/0'
# for id in tedf[tedf['PHQ_Binary']==0]['Participant_ID']:
#   with Image(filename = os.path.join(src, str(id) +'_P_spectrogram.png')) as img:
#     img.noise("gaussian")
#     img.save(filename = os.path.join(dst, str(id) +'aug_P_spectrogram.png'))


In [37]:
#Class Imbalance in the train set is resolved after data augmentation.
p = len(os.listdir('/content/test/1'))
n = len(os.listdir('/content/test/0'))
print(p,n,n/(p+n))

p = len(os.listdir('/content/train/1'))
n = len(os.listdir('/content/train/0'))
print(p,n,n/(p+n))

p = len(os.listdir('/content/test_aug_noise/1'))
n = len(os.listdir('/content/test_aug_noise/0'))
print(p,n,n/(p+n))

p = len(os.listdir('/content/train_aug_noise/1'))
n = len(os.listdir('/content/train_aug_noise/0'))
print(p,n,n/(p+n))

17 39 0.6964285714285714
37 126 0.7730061349693251
51 39 0.43333333333333335
111 126 0.5316455696202531


In [39]:
# https://medium.com/@makcedward/data-augmentation-for-audio-76912b01fdf6
# https://towardsdatascience.com/audio-deep-learning-made-simple-part-3-data-preparation-and-augmentation-24c6e1f6b52

In [40]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [41]:
preproc_func = tensorflow.keras.applications.xception.preprocess_input

In [44]:
#batch size
bs = 32
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preproc_func)
train_gen = train_datagen.flow_from_directory('/content/train_aug_noise', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)
# train_gen = train_datagen.flow_from_directory('/content/train', (224,224), class_mode = 'categorical', batch_size = bs, shuffle=True, seed=42)

print('Class Indices')
print(train_gen.class_indices)

Found 237 images belonging to 2 classes.
Class Indices
{'0': 0, '1': 1}


In [45]:
#batch size
bs = 32
test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preproc_func)
test_gen = test_datagen.flow_from_directory('/content/test_aug_noise', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)
# test_gen = test_datagen.flow_from_directory('/content/test', (224,224), class_mode = 'binary', batch_size = bs, shuffle=True, seed=42)

print('Class Indices')
print(test_gen.class_indices)

Found 90 images belonging to 2 classes.
Class Indices
{'0': 0, '1': 1}


In [42]:
#Custom Callback Function To Save Model Based On A Combined Threshold of train and test accuracy and recall.

import tensorflow as tf
from tensorflow import keras
class RecallAccuracyCallback(keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
      print('Epoch:',epoch, 'Logs:',logs)

      if logs['accuracy']>0.80 and logs['recall']>0.76 and logs['val_accuracy'] > 0.6 and logs['val_recall']>0.8:
        print('Val Accuracy More Than Set Thresholds', logs)
        print('Saving Model','-'*30)
        self.model.save(f'{epoch}_model{str(logs)}.h5')



In [ ]:
#Results Obtained, Models Saved
# 104_model{'loss': 0.4074825346469879, 'accuracy': 0.8374999761581421, 'recall': 0.9130434989929199,  'val_accuracy': 0.6111111044883728, 'val_recall': 0.843137264251709}.h5 'val_loss': 0.6829150915145874,
# 86_model{'loss': 0.4598311483860016, 'accuracy': 0.8374999761581421, 'recall': 0.7702702879905701,  'val_accuracy': 0.6333333253860474, 'val_recall': 0.8039215803146362}.h5 'val_loss': 0.6286814212799072,
# /content/98_model{'loss': 0.39345240592956543, 'accuracy': 0.8581560254096985, 'recall': 0.8529411554336548, , 'val_accuracy': 0.6111111044883728, 'val_recall': 0.843137264251709}.h5

In [85]:
#Recall Metric For Monitoring
r = tf.keras.metrics.Recall()

# Final Custom CNN with Data Augmentation

In [87]:
#Custom CNN 2
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout

model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(224,224,3)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Conv2D(16, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.25))
# model.add(Dense(10, activation=softmax))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#Optional Checkpoints to save models based on different metrics.
from tensorflow.keras.callbacks import ModelCheckpoint
filepath_r = 'saug_weights_best_rmodel.hdf5'
filepath_a = 'saug_weights_best_amodel.hdf5'
filepath_vr = 'saug_weights_best_vrmodel.hdf5'
filepath_va = 'saug_weights_best_vamodel.hdf5'

# checkpoint = ModelCheckpoint(filepath,monitor = 'val_accuracy',verbose = 1, save_best_only=True, mode='max')
checkpoint1 = ModelCheckpoint(filepath_vr,monitor = 'val_recall',verbose = 1, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath_va,monitor = 'val_accuracy',verbose = 1, save_best_only=True, mode='max')
checkpoint3 = ModelCheckpoint(filepath_a,monitor = 'accuracy',verbose = 1, save_best_only=True, mode='max')
checkpoint4 = ModelCheckpoint(filepath_r,monitor = 'recall',verbose = 1, save_best_only=True, mode='max')

optim = tf.keras.optimizers.Adam(
    # learning_rate=0.001,
    # learning_rate=0.00001,
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"
)

model.compile(loss='binary_crossentropy', metrics=['accuracy', r], optimizer=optim)
#With 0.0001 lr.
n_epochs = 1000
num_train = trdf.shape[0]
num_iterations = int(num_train/bs)

# history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations, callbacks =[checkpoint1,checkpoint2,checkpoint3,checkpoint4])
history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations, callbacks = [RecallAccuracyCallback()])
# history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations, callbacks =[checkpoint1,checkpoint2])

Epoch 1/1000
5/5 [==============================] - 2s 253ms/step - loss: 0.7000 - accuracy: 0.5188 - recall: 0.6048 - val_loss: 0.6855 - val_accuracy: 0.5667 - val_recall: 1.0000
Epoch 2/1000
5/5 [==============================] - 1s 136ms/step - loss: 0.6982 - accuracy: 0.4965 - recall: 0.4493 - val_loss: 0.6955 - val_accuracy: 0.4333 - val_recall: 0.0000e+00
Epoch 3/1000
5/5 [==============================] - 1s 139ms/step - loss: 0.6928 - accuracy: 0.5319 - recall: 0.0000e+00 - val_loss: 0.6946 - val_accuracy: 0.4333 - val_recall: 0.0000e+00
Epoch 4/1000
5/5 [==============================] - 1s 144ms/step - loss: 0.6921 - accuracy: 0.5437 - recall: 0.0000e+00 - val_loss: 0.6964 - val_accuracy: 0.4333 - val_recall: 0.0000e+00
Epoch 5/1000
5/5 [==============================] - 1s 143ms/step - loss: 0.6918 - accuracy: 0.5250 - recall: 0.0000e+00 - val_loss: 0.6991 - val_accuracy: 0.4333 - val_recall: 0.0000e+00
Epoch 6/1000
5/5 [==============================] - 1s 143ms/step - loss

KeyboardInterrupt: ignored

In [96]:
#Moving the weights from colab to gdrive
# !mv /content/model_e104_a83_r91_va61_vr84.h5 /content/drive/MyDrive/CV-EmpathNet/
# !mv /content/model_e /content/drive/MyDrive/CV-EmpathNet/
# !mv /content/model85856184.h5 /content/drive/MyDrive/CV-EmpathNet/

#Best Results = 
# 104_model{'loss': 0.4074825346469879, 'accuracy': 0.8374999761581421, 'recall': 0.9130434989929199,  'val_accuracy': 0.6111111044883728, 'val_recall': 0.843137264251709}.h5 'val_loss': 0.6829150915145874,

# Xception with Data Augmentation

In [4]:
r = tf.keras.metrics.Recall()

In [47]:
#Custom Callback Function To Save Xception Model based on  metric thresholds

import tensorflow as tf
from tensorflow import keras
class XceptionRecallAccuracyCallback(keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
      print('Epoch:',epoch, 'Logs:',logs)

      # if logs['accuracy']>0.80 and logs['recall']>0.76 and logs['val_accuracy'] > 0.6 and logs['val_recall']>0.8:
      if logs['accuracy']>0.80 and logs['recall']>0.7 and logs['val_accuracy'] > 0.8 and logs['val_recall']>0.6:
        print('Val Accuracy More Than Set Thresholds', logs)
        print('Saving Model','-'*30)
        self.model.save(f'/content/drive/MyDrive/CV-EmpathNet/XceptionNetWeights/{epoch}_model{str(logs)}.h5')



In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.xception import Xception



pretrained_model = Xception(include_top=False, weights='imagenet', input_shape=(224,224,3))
pretrained_model.trainable = False
model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))


#Optional Checkpoints to save model  based on metrics
from tensorflow.keras.callbacks import ModelCheckpoint
filepath_r = 'saug_weights_best_rmodel.hdf5'
filepath_a = 'saug_weights_best_amodel.hdf5'
filepath_vr = 'saug_weights_best_vrmodel.hdf5'
filepath_va = 'saug_weights_best_vamodel.hdf5'

checkpoint1 = ModelCheckpoint(filepath_vr,monitor = 'val_recall_25',verbose = 1, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath_va,monitor = 'val_accuracy',verbose = 1, save_best_only=True, mode='max')
checkpoint3 = ModelCheckpoint(filepath_a,monitor = 'accuracy',verbose = 1, save_best_only=True, mode='max')
checkpoint4 = ModelCheckpoint(filepath_r,monitor = 'recall_25',verbose = 1, save_best_only=True, mode='max')

#With 0.0001 lr.
optim = tf.keras.optimizers.Adam(
    # learning_rate=0.001,
    # learning_rate=0.00001,
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"
)

model.compile(loss='binary_crossentropy', metrics=['accuracy', r], optimizer=optim)
n_epochs = 1000
num_train = trdf.shape[0]
num_iterations = int(num_train/bs)

# history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations, callbacks =[checkpoint1,checkpoint2,checkpoint3,checkpoint4])
# history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations, callbacks =[checkpoint1,checkpoint2])
history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations, callbacks = [XceptionRecallAccuracyCallback()])

Epoch 1/1000
5/5 [==============================] - 4s 352ms/step - loss: 0.6872 - accuracy: 0.6000 - recall: 0.4758 - val_loss: 0.7047 - val_accuracy: 0.4333 - val_recall: 0.0000e+00
Epoch 2/1000
5/5 [==============================] - 1s 136ms/step - loss: 0.6844 - accuracy: 0.5437 - recall: 0.0000e+00 - val_loss: 0.7058 - val_accuracy: 0.4333 - val_recall: 0.0000e+00
Epoch 3/1000
5/5 [==============================] - 1s 125ms/step - loss: 0.6848 - accuracy: 0.5248 - recall: 0.0290 - val_loss: 0.6873 - val_accuracy: 0.4778 - val_recall: 0.0784
Epoch 4/1000
5/5 [==============================] - 1s 128ms/step - loss: 0.6802 - accuracy: 0.6383 - recall: 0.3000 - val_loss: 0.6771 - val_accuracy: 0.7222 - val_recall: 0.5294
Epoch 5/1000
5/5 [==============================] - 1s 126ms/step - loss: 0.6812 - accuracy: 0.7163 - recall: 0.5873 - val_loss: 0.6790 - val_accuracy: 0.5778 - val_recall: 0.2549
Epoch 6/1000
5/5 [==============================] - 1s 134ms/step - loss: 0.6652 - accur

KeyboardInterrupt: ignored

In [ ]:
#Whole Model Trainable
for layer in model.layers:
  layer.trainable = True

#Whole Model Trainable
for layer in model.layers:
  print(layer.trainable)

True
True
True


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath_r = 'saug_weights_best_rmodel.hdf5'
filepath_a = 'saug_weights_best_amodel.hdf5'
filepath_vr = 'saug_weights_best_vrmodel.hdf5'
filepath_va = 'saug_weights_best_vamodel.hdf5'

# checkpoint = ModelCheckpoint(filepath,monitor = 'val_accuracy',verbose = 1, save_best_only=True, mode='max')
# checkpoint1 = ModelCheckpoint(filepath_vr,monitor = 'val_recall_25',verbose = 1, save_best_only=True, mode='max')
# checkpoint2 = ModelCheckpoint(filepath_va,monitor = 'val_accuracy',verbose = 1, save_best_only=True, mode='max')

checkpoint3 = ModelCheckpoint(filepath_a,monitor = 'accuracy',verbose = 1, save_best_only=True, mode='max')
# checkpoint4 = ModelCheckpoint(filepath_r,monitor = 'recall_25',verbose = 1, save_best_only=True, mode='max')

In [ ]:
# r = tf.keras.metrics.Recall()


In [ ]:
#Fine Tuning

optim = tf.keras.optimizers.Adam(
    # learning_rate=0.001,
    learning_rate=0.00001,
    # learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"
)


# model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
# model.compile(loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Recall()], optimizer=optim)
model.compile(loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Recall()], optimizer=optim)
# model.compile(loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Recall()], optimizer=optim)
#With 0.0001 lr.
n_epochs = 1000
num_train = trdf.shape[0]
num_iterations = int(num_train/bs)

history = model.fit(train_gen, epochs=n_epochs, validation_data=test_gen, steps_per_epoch=num_iterations)

Epoch 1/1000
5/5 [==============================] - 2s 193ms/step - loss: 0.6886 - accuracy: 0.6313 - recall_1: 0.2188 - val_loss: 0.6901 - val_accuracy: 0.5342 - val_recall_1: 0.0000e+00
Epoch 2/1000
5/5 [==============================] - 1s 112ms/step - loss: 0.6694 - accuracy: 0.6397 - recall_1: 0.0000e+00 - val_loss: 0.7268 - val_accuracy: 0.5342 - val_recall_1: 0.0000e+00
Epoch 3/1000
5/5 [==============================] - 1s 114ms/step - loss: 0.6742 - accuracy: 0.6187 - recall_1: 0.0000e+00 - val_loss: 0.7304 - val_accuracy: 0.5342 - val_recall_1: 0.0000e+00
Epoch 4/1000
5/5 [==============================] - 1s 108ms/step - loss: 0.6570 - accuracy: 0.6397 - recall_1: 0.0000e+00 - val_loss: 0.7149 - val_accuracy: 0.5342 - val_recall_1: 0.0000e+00
Epoch 5/1000
5/5 [==============================] - 1s 101ms/step - loss: 0.6603 - accuracy: 0.6250 - recall_1: 0.0000e+00 - val_loss: 0.7143 - val_accuracy: 0.5342 - val_recall_1: 0.0000e+00
Epoch 6/1000
5/5 [==========================

KeyboardInterrupt: ignored